In [1]:
import numpy as np
from scipy.stats import unitary_group
from scipy.linalg import eig, expm
from scipy.optimize import minimize
from functools import reduce 
from xmps.spin import U4

# tm_left:
"""
                   0      0      j
                   | -U2- |      |
A[i, σ1,σ2, j] =   |      | -U1- |
                   |      |      |
                   i      σ1      σ2
                   
"""

# tm_right:
"""
                   i      0      0
                   |      | -U2- |
A[i, σ1,σ2, j] =   | -U1- |      |
                   |      |      |
                   σ1     σ2     j
"""

Z = np.array([
    [1,0],
    [0,-1]
])

X = np.array([
    [0,1],
    [1,0]
])

I = np.eye(2)

def tensor(tensors):
    return reduce(lambda t1,t2: np.kron(t1,t2), tensors)


In [4]:
def state(U2,U1):
    return np.einsum(
        U1, [1,2,4,5],
        U2, [0,4,6,7],
        U2, [5,3,8,9],
        [0,1,2,3,6,7,8,9]
    )

def tm_right(U2, U1):
    """
    A[i,σ1,σ2,j] = Σα U1[σ1,σ2,i,α]U2[α,j,0,0]
    """
    
    return np.transpose(np.tensordot(
        U1.reshape(2,2,2,2),
        U2[:,0].reshape(2,2),
        (3,0)
    ),(2,0,1,3)).reshape(2,4,2)

def tm_left(U2, U1):
    """
    A[i,σ1,σ2,j] = Σα U1[σ1,σ2,α,j]U2[i,α,0,0]
    """
    return np.transpose(np.tensordot(
        U1.reshape(2,2,2,2),
        U2[:,0].reshape(2,2),
        (2,1)
    ),(3,0,1,2)).reshape(2,4,2)

def single_transfer_matrix(s, s̄):
    s̄ = s̄.conj()
    ss̄ = np.transpose(np.tensordot(s̄,s,(1,1)),(0,2,1,3)).reshape(4,4)
    return ss̄

def right_env(U2,U1,Ū2,Ū1):
    alt_state = tm_right(U2, U1)
    alt_state_bar = tm_right(Ū2, Ū1)
    tm = single_transfer_matrix(alt_state, alt_state_bar)
    η, v = eig(tm)
    vr = v[:,np.argmax(np.abs(η))].reshape(2,2)
    return vr, η

def left_env(U2,U1,Ū2,Ū1):
    alt_state = tm_left(U2, U1)
    alt_state_bar = tm_left(Ū2, Ū1)
    tm = single_transfer_matrix(alt_state, alt_state_bar)
    η, v = eig(tm.T)
    vl = v[:,np.argmax(np.abs(η))].reshape(2,2)
    return vl, η

def approx_adjusted_env(U2, Ū2, vr):
    env = np.einsum(
        Ū2.conj().T[0,:].reshape(2,2),[0,1],
        vr.T,[1,3],
        U2[:,0].reshape(2,2),[2,3],
        )
    return env


def overlap(U2,U1,Ū2,Ū1, Ut):
    s = state(U2.reshape(2,2,2,2), U1.reshape(2,2,2,2)).reshape(16,16)
    s̄ = state(Ū2.reshape(2,2,2,2), Ū1.reshape(2,2,2,2)).reshape(16,16)
    vl, _ = left_env(U2,U1,Ū2,Ū1)
    vr, _ = right_env(U2,U1,Ū2,Ū1)
    
    env_norm = np.linalg.norm(approx_adjusted_env(U2, Ū2, vr))
    
    middle = tensor([vr,Ut,vl])

    unitary = s̄.conj().T @ middle @ s
    o = (2*np.abs(unitary[0,0]))**2
    return o


In [263]:
def Hamiltonian(J, g):
    ZZ = np.kron(Z,Z)
    XX = 0.5*(np.kron(I,X) + np.kron(X,I))
    return J*ZZ + g*XX

def overlap(U2,U1, Ū2,Ū1, Ut):
    s = state(U2.reshape(2,2,2,2), U1.reshape(2,2,2,2)).reshape(16,16)
    s̄ = state(Ū2.reshape(2,2,2,2), Ū1.reshape(2,2,2,2)).reshape(16,16)
    vl, _ = left_env(U2,U1,Ū2,Ū1)
    vr, _ = right_env(U2,U1,Ū2,Ū1)
    
    middle = tensor([vr,Ut,vl])

    unitary = s̄.conj().T @ middle @ s
    o = (2*np.abs(unitary[0,0]))**2
    return o

def obj(param, U2,U1, Ut):
    Ū2 = U4(param[:15])
    Ū1 = U4(param[15:])
    return -overlap(U2,U1, Ū2,Ū1, Ut)


init_param = np.random.rand(30)
U2 = U4(init_param[:15])
U1 = U4(init_param[15:])
Ut = expm(-1j * Hamiltonian(1,1) * 0.0)

res = minimize(
        obj,
        x0 = init_param,
        method = "Nelder-Mead",
        args = (U2,U1,Ut)
    )

print(res.fun)
print(res.x - init_param)

obj(init_param, U2,U1,Ut)


-2.9210512643737094
[-0.01877387 -0.20590864  0.02098783 -0.1883981   0.11633405 -0.09752193
  0.32501491 -0.04169835 -0.33614282 -0.41622419 -0.28497304 -0.0907983
  0.57979352 -0.1050277   0.22258675  1.23266039 -1.65533096  0.03279119
 -0.72338732  0.52322373  0.25580002 -0.20535053 -0.95739525  0.24381527
 -0.06254336  0.15657934  0.09159621 -0.07278437 -0.04082857  0.07348164]


-1.0000000000000004

In [266]:
U2 = U4(init_param[:15])
U1 = U4(init_param[15:])

Ū2 = U4(res.x[:15])
Ū1 = U4(res.x[15:])

s = state(U2.reshape(2,2,2,2), U1.reshape(2,2,2,2)).reshape(16,16)
s̄ = state(Ū2.reshape(2,2,2,2), Ū1.reshape(2,2,2,2)).reshape(16,16)
vl, ηl = left_env(U2,U1,Ū2,Ū1)
vr, ηr = right_env(U2,U1,Ū2,Ū1)
#vr = rotate_to_hermitian(vr)/np.sign(vr[0,0])

middle = tensor([vl ,Ut, vr])

unitary = s.conj().T @ middle @ s
o = (2*np.abs(unitary[0,0]))**2

In [267]:
obj(init_param, U2, U1, Ut)

-1.0000000000000004

In [265]:
(s̄.conj().T@s)[0,0]

(-0.0819978956177038+0.029979202990769796j)

(0.9844829541064992+0.06009181401984644j)

In [272]:
vr

np.linalg.norm(rotate_to_hermitian(vr) / np.sign(vr[0,0]))

1.0000000000000002

In [269]:
alt_state = tm_left(U2, U1)
alt_state_bar = tm_left(Ū2, Ū1)
tm = single_transfer_matrix(alt_state, alt_state_bar)

print(np.round(tm,3))

[[0.123+0.077j 0.107+0.085j 0.086+0.009j 0.126-0.011j]
 [0.313+0.111j 0.34 +0.115j 0.221-0.029j 0.252-0.037j]
 [0.069+0.143j 0.055+0.16j  0.071+0.044j 0.134+0.087j]
 [0.221+0.275j 0.275+0.347j 0.207+0.074j 0.29 +0.165j]]


In [5]:
U2 = unitary_group.rvs(4)
U1 = unitary_group.rvs(4)

left_env(U2, U1, U2, U1)

(array([[7.07106781e-01+3.64086364e-25j, 1.49649444e-16-5.33586109e-17j],
        [1.49649437e-16+5.33586109e-17j, 7.07106781e-01+0.00000000e+00j]]),
 array([ 1.00000000e+00-1.21129717e-24j,  3.86780049e-17+3.44201082e-19j,
        -2.83394782e-17-6.51952706e-17j, -6.66250213e-17+6.48510696e-17j]))